In [1]:
import sys
import os
from rouge_metric import PyRouge
import xmltodict
import untangle

import pandas as pd
import numpy as np

import nltk
from nltk import pos_tag
from nltk import sent_tokenize
from nltk import word_tokenize 
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from num2words import num2words 
import re
import copy

In [2]:
sys.path.insert(0, "C:/Users/Lenovo/Downloads/scisumm/")
BASE_DIR = "C:/Users/Lenovo/Downloads/scisumm/"

In [3]:
articles = []
summaries = []

In [4]:
for folder in os.listdir(BASE_DIR):
    for subfolder in os.listdir(BASE_DIR + folder):
        for filename in os.listdir(BASE_DIR + folder + "/" + subfolder):
            if(subfolder == "Reference_XML"):
                print("file", filename)
                xml_dict = xmltodict.parse(open(os.path.join(BASE_DIR + folder + "/" + subfolder + "/" + filename),encoding='utf-8', errors='ignore').read())
                articles.append(xml_dict)
            else:
                #print(os.path.join(BASE_DIR + folder + "/" + subfolder + "/" + filename))
                print("Summary", filename)
                summary = open(os.path.join(BASE_DIR + folder + "/" + subfolder + "/" + filename), encoding='utf-8', errors='ignore').read()
                summaries.append(summary)            

file C00-2123.xml
Summary C00-2123.community.txt
file C02-1025.xml
Summary C02-1025.community.txt
file C04-1089.xml
Summary C04-1089.community.txt
file C08-1098.xml
Summary C08-1098.community.txt
file C10-1045.xml
Summary C10-1045.community.txt
file C90-2039.xml
Summary C90-2039.community.txt
file C94-2154.xml
Summary C94-2154.community.txt
file D10-1083.xml
Summary D10-1083.community.txt
file E03-1020.xml
Summary E03-1020.community.txt
file E09-2008.xml
Summary E09-2008.community.txt
file H05-1115.xml
Summary H05-1115.community.txt
file H89-2014.xml
Summary H89-2014.community.txt
file I05-5011.xml
Summary I05-5011.community.txt
file J00-3003.xml
Summary J00-3003.community.txt
file J96-3004.xml
Summary J96-3004.community.txt
file J98-2005.xml
Summary J98-2005.community.txt
file N01-1011.xml
Summary N01-1011.summary.txt
file N04-1038.xml
Summary N04-1038.community.txt
file N06-2049.xml
Summary N06-2049-community.txt
file P05-1004.xml
Summary P05-1004.community.txt
file P05-1053.xml
Summ

# Data Preparation

In [5]:
# Splitting the summary into sentences and removig empty string
summaries_split = []

for i in range(len(summaries)):
    summaries_split.insert(i, summaries[i].split("\n"))
    while True:
        try:
            summaries_split[i].remove("")
        except ValueError:
            break

In [6]:
papers = []

'''
Produce Data Structure from xlm Data

Exceptions occur, if one of the xml-tags is empty
'''
for i in range(len(articles)):
    paper = []
    for key in articles[0]['PAPER'].keys():
        if(key == 'S'):
            title = articles[0]['PAPER']['S']['#text']
            #append title to paper
            paper.insert(0, title)
        
        text_sentences = []

        # Shuffle through all sentences in the abstract
        if(key == 'ABSTRACT'):
            try:
                for j in range(len(articles[i]['PAPER']['ABSTRACT']['S'])):
                        sentence = articles[i]['PAPER']['ABSTRACT']['S'][j]['#text']
                        text_sentences.append(sentence)
            except:
                print("Exception in Abstract:", "i", i)


        # Shuffle through all sentences in the sections - introduction, methods, results, ...
        # articles[0]['PAPER']['SECTION'][0]['S'][0]['#text']
        if(key == 'SECTION'):
            try:
                for j in range(len(articles[i]['PAPER']['SECTION'])):
                        for k in range(len(articles[i]['PAPER']['SECTION'][j]['S'])):
                            sentence = articles[i]['PAPER']['SECTION'][j]['S'][k]['#text']
                            text_sentences.append(sentence)
            except:
                print("Exception in Text:", "i", i, "j", j)
            
        #append sentences to paper
        paper.insert(1, text_sentences)
        paper.insert(2, summaries_split[i])

    papers.append(paper)

Exception in Text: i 5 j 6
Exception in Text: i 10 j 7
Exception in Text: i 17 j 2
Exception in Text: i 18 j 5
Exception in Text: i 23 j 3
Exception in Text: i 28 j 5
Exception in Abstract: i 29


# HOW TO HANDLE THE DATA NOW?

In [7]:
'''
papers = list of papers
    Sub-Indices:
    [0] - title
    [1] - abstract
    [2] - text
'''

# Access the title of the first paper
papers[0][0]

'Word Re-ordering and DP-based Search in Statistical Machine Translation'

In [8]:
# Access the text of the first paper
papers[0][1]

# Access the first sentence of the text of the first paper
papers[0][1][0]

'The goal of machine translation is the translation of a text given in some source language into a target language.'

In [9]:
# Access the summary of the first paper
papers[0][2]

# Access the first sentence of the summary of the first paper
papers[0][2][0]

print(papers[0][1])

['The goal of machine translation is the translation of a text given in some source language into a target language.', 'We are given a source string fJ 1 = f1:::fj :::fJ of length J, which is to be translated into a target string eI 1 = e1:::ei:::eI of length I. Among all possible target strings, we will choose the string with the highest probability: ^eI 1 = arg max eI 1 fPr(eI 1jfJ 1 )g = arg max eI 1 fPr(eI 1) Pr(fJ 1 jeI 1)g : (1) The argmax operation denotes the search problem, i.e. the generation of the output sentence in the target language.', 'Pr(eI 1) is the language model of the target language, whereas Pr(fJ 1 jeI1) is the transla tion model.', 'Our approach uses word-to-word dependencies between source and target words.', 'The model is often further restricted so that each source word is assigned to exactly one target word (Brown et al., 1993; Ney et al., 2000).', 'These alignment models are similar to the concept of hidden Markov models (HMM) in speech recognition.', 'The 

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [11]:
#All the functions for TF-IDF and SMMRY calculations

#Creating unique words list
def find_unique_words(sentences): 
    unique_words = list()
    for sentence in sentences:
        words = word_tokenize(sentence) 
        for word in words:
            if word not in unique_words:
                unique_words.append(word)    
    return unique_words  
    
#Creating frequency matrix of the words in each sentence
def create_freq_matrix(sentences):
    freq = dict()
    for sentence in sentences:
        words = word_tokenize(sentence)
        freq_dict = dict()

        for word in words:
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1

        freq[sentence] = freq_dict   
    return freq
    
#TF
def find_tf(freq):
    tf = dict()
    for sentence in freq:
        words = word_tokenize(sentence)
        tf_dict = dict()
        for word in words:
            tf_dict[word] = freq[sentence][word] /len(words)
        tf[sentence] = tf_dict
    return tf
    
#IDF
def find_idf(uninique_words, sentences):
    idf = dict()
    for word in unique_words:
        for sentence in sentences:
            words = word_tokenize(sentence) 
            if word in set(words):
                if word in idf:
                    idf[word] += 1
                else:
                    idf[word] = 1         
    for word in idf:
        idf[word] = np.log(len(sentences)/idf[word])
    return idf

#Combining TF IDF
def find_tf_idf(tf, idf):
    tf_idf = dict()
    for sentence in tf:
        words = word_tokenize(sentence)
        tf_idf_dict = dict()
        for word in words:
            tf_idf_dict[word] = tf[sentence][word] * idf[word]
        tf_idf[sentence] = tf_idf_dict
    return tf_idf

#Finding sentences with highest score
def find_top_n_sentences(tf_idf, n, lookup):
    #Finding n sentences with higesht score
    answer_head = list()
    answer_value = list()
    for key, value in tf_idf.items():
        answer_value.append(sum(value.values()))
        answer_head.append(key)

    answ = sorted(zip(answer_value, answer_head), reverse=True)[:n]

    #Getting not edited versions of the sentences
    val, sen = zip(*answ)
    for i in range(len(val)):
        answ[i] = lookup[sen[i]]
    return(answ)
    
    
#Creating an array from the dictionary (including all the words)
def create_tf_idf_array_from_dict(tf_idf, sentences, unique_words):
    i = len(unique_words)
    arr = np.zeros((len(sentences), len(unique_words)))

    for i, (key, sentence) in enumerate(tf_idf.items()):
        for j, (word, value) in enumerate(sentence.items()):
            idx = unique_words.index(word)
            arr[i][idx] = value
    return arr

#Creating similarity matrix        
def find_similarity_matrix(X):
    cos_sim = np.dot(X,X.T)
    np.fill_diagonal(cos_sim, 0) 
      
    for index in range(len(cos_sim)):
        if cos_sim[index].sum() == 0:
            continue
        else:
            cos_sim[index] /= cos_sim[index].sum()
    return cos_sim

#Finding the textrank of the sentence (this function is borrowed from https://github.com/foprel/smmry/blob/master/app.py)
def find_textrank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P

def find_max_n_values(sentences_original, text_rank, n):
    answ = sorted(zip(text_rank, sentences_original), reverse=True)[:n]
    val, sentence = zip(*answ)
    return sentence

In [12]:
def preprocessing(sentences, sentences_original):
    for index, sentence in enumerate(sentences):
        
        #Making Lowercase
        sentence = sentence.lower()
        
        #Removing everything that is not a letter (punctuations, digits, unknown symbols)
        sentence = re.sub(r'[^a-zA-Z ]','',sentence) 
        
        words_list = word_tokenize(sentence)
        filtered_words = words_list
        
        #Removing stopwords
        filtered_words = [word for word in words_list if word not in set(stopwords.words("english"))] 
        
        #Removing one/two letter words
        filtered_words = [word for word in filtered_words if len(word)>2]
                
        if (len(filtered_words)<=2 or (not filtered_words)):
            sentence_to_be_removed = sentences_original[index]
            sentences.pop(index)
            sentences_original.pop(index)
        else:

            #Converting numbers to text
            #filtered_words = [num2words(word, lang='en') if word.isdigit() else word for word in filtered_words]

            #Remove not english words
            #filtered_words = [w for w in filtered_words if w in words.words()]

            #Remove digits
            #filtered_words = [word for word in filtered_words if not word.isdigit()]

            #Stemming (would be nice to try lematisation or both)
            ps = PorterStemmer()
            filtered_words = [ps.stem(word) for word in filtered_words] 

            sentence = ' '.join(filtered_words)   
            sentences[index] = sentence 
        
    return sentences, sentences_original
        
        
def filtering_tags(preprocessed_tokens, pos=['NN', 'ADJ']):
    
    for i, sentence in enumerate(sentences):
        words_list = word_tokenize(sentence)
        
        filtered_words = []
        for index, s in enumerate(words_list):
            s = pos_tag([s])
            if s[0][1] in pos:
                filtered_words.append(s[0][0])      
        
        sentence = ' '.join(filtered_words)   
        sentences[i] = sentence   
        
    return sentences

In [13]:
# Main
tf_idf_score = list()
smmry_score = list()

tf_idf_predicted = list()
smmry_predicted = list()

for idx, article in enumerate(papers): #enumerate(papers[0:10]): #Select all papers
    n = len(papers[idx][2]) #number of sentences to select for summarisation
    sentences_original = copy.deepcopy(article[1]) 
    sentences = copy.deepcopy(article[1])
    
    #Data pre-processing
    sentences, sentences_original = preprocessing(sentences, sentences_original) 
    sentences_smmry = filtering_tags(sentences)
    
    #TF-IDF part
    lookup = dict()
    for i in range(len(sentences)):
        lookup[sentences[i]] = sentences_original[i]

    unique_words = find_unique_words(sentences)
    freq_matrix = create_freq_matrix(sentences)
    
    tf = find_tf(freq_matrix)
    idf = find_idf(unique_words, sentences)
    tf_idf = find_tf_idf(tf, idf)
    top_n_sentences_tf_idf = find_top_n_sentences(tf_idf, n, lookup)
    tf_idf_predicted.append(top_n_sentences_tf_idf)

    score=0
    for i, sent in enumerate(top_n_sentences_tf_idf):
        if sent in papers[idx][2]:
            score+=1/n
    tf_idf_score.append(score)

    #SMMRY part
    sentences = sentences_smmry
    lookup = dict()
    for i in range(len(sentences)):
        lookup[sentences[i]] = sentences_original[i]

    unique_words = find_unique_words(sentences)
    freq_matrix = create_freq_matrix(sentences)
    
    tf = find_tf(freq_matrix)
    idf = find_idf(unique_words, sentences)
    tf_idf = find_tf_idf(tf, idf)
    
    tf_idf_array = create_tf_idf_array_from_dict(tf_idf, sentences, unique_words)    
    similarity_matrix = find_similarity_matrix(tf_idf_array)
    textrank = find_textrank(similarity_matrix)

    top_n_sentences_textrank = find_max_n_values(sentences_original, textrank, n)
    smmry_predicted.append(top_n_sentences_textrank)

    score=0
    for i, sent in enumerate(top_n_sentences_textrank):
        if sent in papers[idx][2]:
            score+=1/n
    smmry_score.append(score)

In [14]:
#Evaluation

#Accuracy
print("TF-IDF", np.mean(tf_idf_score))
print("SMMRY", np.mean(smmry_score),"\n")

#ROUGE-1, ROUGE-2 and ROUGE-L metrics
rouge = PyRouge(rouge_n=(1, 2), rouge_l=True)
print("ROUGE metrics:")

tf_idf_rouge_1_r = list(); tf_idf_rouge_1_p = list(); tf_idf_rouge_1_f = list()
tf_idf_rouge_2_r = list(); tf_idf_rouge_2_p = list(); tf_idf_rouge_2_f = list()
tf_idf_rouge_l_r = list(); tf_idf_rouge_l_p = list(); tf_idf_rouge_l_f = list()

smmry_rouge_1_r = list(); smmry_rouge_1_p = list(); smmry_rouge_1_f = list()
smmry_rouge_2_r = list(); smmry_rouge_2_p = list(); smmry_rouge_2_f = list()
smmry_rouge_l_r = list(); smmry_rouge_l_p = list(); smmry_rouge_l_f = list()

for i in range(len(tf_idf_predicted)):
    
    scores_tf_idf = rouge.evaluate(tf_idf_predicted[i], papers[i][2])
    tf_idf_rouge_1_r.append(scores_tf_idf['rouge-1']['r'])
    tf_idf_rouge_2_r.append(scores_tf_idf['rouge-2']['r'])
    tf_idf_rouge_l_r.append(scores_tf_idf['rouge-l']['r'])
    
    tf_idf_rouge_1_p.append(scores_tf_idf['rouge-1']['p'])
    tf_idf_rouge_2_p.append(scores_tf_idf['rouge-2']['p'])
    tf_idf_rouge_l_p.append(scores_tf_idf['rouge-l']['p'])
    
    tf_idf_rouge_1_f.append(scores_tf_idf['rouge-1']['f'])
    tf_idf_rouge_2_f.append(scores_tf_idf['rouge-2']['f'])
    tf_idf_rouge_l_f.append(scores_tf_idf['rouge-l']['f'])
    
    
    scores_smmry = rouge.evaluate(smmry_predicted[i], papers[i][2])
    smmry_rouge_1_r.append(scores_smmry['rouge-1']['r'])
    smmry_rouge_2_r.append(scores_smmry['rouge-2']['r'])
    smmry_rouge_l_r.append(scores_smmry['rouge-l']['r'])
    
    smmry_rouge_1_p.append(scores_smmry['rouge-1']['p'])
    smmry_rouge_2_p.append(scores_smmry['rouge-2']['p'])
    smmry_rouge_l_p.append(scores_smmry['rouge-l']['p'])
    
    smmry_rouge_1_f.append(scores_smmry['rouge-1']['f'])
    smmry_rouge_2_f.append(scores_smmry['rouge-2']['f'])
    smmry_rouge_l_f.append(scores_smmry['rouge-l']['f'])
      
print("TF-IDF rouge 1 score: ", "r:", np.mean(tf_idf_rouge_1_r)," p:", np.mean(tf_idf_rouge_1_r), "f:", np.mean(tf_idf_rouge_1_f)) 
print("TF-IDF rouge 2 score: ", "r:", np.mean(tf_idf_rouge_2_r)," p:", np.mean(tf_idf_rouge_2_r), "f:", np.mean(tf_idf_rouge_2_f))
print("TF-IDF rouge L score: ", "r:", np.mean(tf_idf_rouge_l_r)," p:", np.mean(tf_idf_rouge_l_r), "f:", np.mean(tf_idf_rouge_l_f))
print("SMMRY rouge 1 score: ", "r:", np.mean(smmry_rouge_1_r)," p:", np.mean(smmry_rouge_1_r), "f:", np.mean(smmry_rouge_1_f)) 
print("SMMRY rouge 2 score: ", "r:", np.mean(smmry_rouge_2_r)," p:", np.mean(smmry_rouge_2_r), "f:", np.mean(smmry_rouge_2_f))
print("SMMRY rouge L score: ", "r:", np.mean(smmry_rouge_l_r)," p:", np.mean(smmry_rouge_l_r), "f:", np.mean(smmry_rouge_l_f))

TF-IDF 0.02624724178521452
SMMRY 0.07782546052279816 

ROUGE metrics:
TF-IDF rouge 1 score:  r: 0.0333653291288336  p: 0.0333653291288336 f: 0.002131200632425981
TF-IDF rouge 2 score:  r: 0.0  p: 0.0 f: 0.0
TF-IDF rouge L score:  r: 0.0333653291288336  p: 0.0333653291288336 f: 0.002131200632425981
SMMRY rouge 1 score:  r: 0.02206080145664252  p: 0.02206080145664252 f: 0.0018426351354706373
SMMRY rouge 2 score:  r: 0.0  p: 0.0 f: 0.0
SMMRY rouge L score:  r: 0.02206080145664252  p: 0.02206080145664252 f: 0.0018426351354706373


In [15]:
#Accuracy of TF-IDF from sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

score_list = list()
for p in papers:

    #Creating TF-IDF vector
    tf_idf_vect = TfidfVectorizer()
    X_train_tf_idf = tf_idf_vect.fit_transform(p[1])
    terms = tf_idf_vect.get_feature_names()
    
    #Converting it to pandas dataframe
    df = pd.DataFrame(X_train_tf_idf.toarray(), columns=terms, index=p[1])
    
    #Summing values of each sentence
    count = pd.DataFrame(df[df!=0].count(axis=1))
    df = pd.DataFrame(df.sum(axis=1))
    df.columns = ['score']
    df["count"] = count
    df = pd.DataFrame(df.sort_values(by="score", ascending=False))
    
    #Calculating accuracy
    score = 0
    for item in df.iloc[0:10].index:
        if item in p[2]:
            score+=1
    score/=len(p[2])
    score_list.append(score)

print("Accuracy of TF-IDF from sklearn:", np.mean(score_list))

Accuracy of TF-IDF from sklearn: 0.0459185287715282
